In [1]:
import urllib.request
f = urllib.request.urlopen("https://www.imgt.org/IMGTrepertoire/index.php?section=LocusGenes&repertoire=genetable&species=human&group=TRBV")
file = f.read().decode("utf-8")

In [2]:
from bs4 import BeautifulSoup
 
soup = BeautifulSoup(file, 'lxml')

In [4]:
import pandas as pd
url = r'https://www.imgt.org/IMGTrepertoire/index.php?section=LocusGenes&repertoire=genetable&species=human&group=TRBV'
tables = pd.read_html(url) # Returns list of all tables on page
sp500_table = tables # Select table of interest

In [17]:
pd.DataFrame(sp500_table[0])['IMGT/LIGM-DB reference sequences']#['Secondary accession numbers']

,Clone names,Clone names.1,Clone names.2,Clone names.3,Accession numbers,Accession numbers.1,Accession numbers.2,Accession numbers.3,Positions in the sequence (L-V-GENE-UNIT),Positions in the sequence (L-V-GENE-UNIT).1,Positions in the sequence (L-V-GENE-UNIT).2,Positions in the sequence (L-V-GENE-UNIT).3,Secondary accession numbers,Secondary accession numbers.1,Secondary accession numbers.2,Secondary accession numbers.3
0,NaN,NaN,NaN,NaN,V1,NaN,NaN,MAP,NaN,NaN,NaN,NaN,L36092,[11],(37),NaN
1,NaN,NaN,NaN,NaN,V2,NaN,NaN,MAP,NaN,NaN,NaN,NaN,L36092,[11],(37),NaN
2,NaN,NaN,NaN,NaN,Vbeta23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,M62379,[19],(41),#c
3,NaN,NaN,NaN,NaN,Vbeta22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,M64351,[20],(41),#c
4,NaN,NaN,NaN,NaN,V9S1,NaN,NaN,MAP,NaN,NaN,NaN,NaN,U07977,[14],NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
278,NaN,NaN,NaN,NaN,CM000669.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,IMGT000021,NaN,(37)(41),NaN
279,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,L36092,[11],(37),NaN
280,NaN,NaN,NaN,NaN,CM000669.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,IMGT000021,NaN,(37),NaN
281,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,L36092,[11],(37)(41),NaN


In [27]:
import urllib.request
from pprint import pprint
from html_table_parser.parser import HTMLTableParser


def url_get_contents(url):
    """ Opens a website and read its binary contents (HTTP Response Body) """
    req = urllib.request.Request(url=url)
    f = urllib.request.urlopen(req)
    return f.read()


def main():
    url = 'https://www.imgt.org/IMGTrepertoire/index.php?section=LocusGenes&repertoire=genetable&species=human&group=TRBV'
    xhtml = url_get_contents(url).decode('utf-8')

    p = HTMLTableParser()
    p.feed(xhtml)
    pprint(p.tables)


#if __name__ == '__main__':
    #res = main()

In [104]:
with open('/home/mrred/Рабочий стол/Работа/t_cells/Allign/RefSeq/ref_data.xml', 'w') as fl:
    fl.write(soup.table.prettify())

In [11]:
from lxml import objectify
import xml.etree.ElementTree as ET
import pandas as pd

xml_data = open('/home/mrred/Рабочий стол/Работа/t_cells/Allign/ref_data.xml', 'r').read()  # Read file
root = ET.XML(xml_data)  # Parse XML

data = []
cols = []
for i in range(len(root.getchildren())):
    child = root.getchildren()[i]
    data.append([subchild.text for subchild in child.getchildren()])
    cols.append(child.tag)

df = pd.DataFrame(data).T  # Create DataFrame and transpose it
df.columns = cols

AttributeError: 'xml.etree.ElementTree.Element' object has no attribute 'getchildren'

In [12]:
import xml.etree.ElementTree as ET

tree = ET.parse('/home/mrred/Рабочий стол/Работа/t_cells/Allign/ref_data.xml')
root = tree.getroot()

In [33]:
import xmltodict
import pandas as pd

xml_data = open('/home/mrred/Рабочий стол/Работа/t_cells/Allign/ref_data.xml', 'r').read()  # Read data
xmlDict = xmltodict.parse(xml_data)

In [147]:
import urllib.request
from bs4 import BeautifulSoup
import xmltodict
import json
import os

def TableParser(name):
    path = os.getcwd()
    
    f = urllib.request.urlopen(f"https://www.imgt.org/IMGTrepertoire/index.php?section=LocusGenes&repertoire=genetable&species=human&group={name}")
    file = f.read().decode("utf-8")
    
    soup = BeautifulSoup(file, 'lxml')
    
    with open(f'{path}/RefSeq/Helpers/{name}.xml', 'w') as fl:
        fl.write(soup.table.prettify())

    xml_data = open(f'{path}/RefSeq/Helpers/{name}.xml', 'r').read()
    xmlDict = xmltodict.parse(xml_data)
    
    checker = ['']
    rs = []
    for i in xmlDict['table']['tr'][2:]:
        dct = {}
        for j in i['td']:
            if j['@class'] == 'allele_note':
                lst = []
                allele_link = 'https://www.imgt.org' + j['a']['@href'].replace(' ', '%20')
                allele_name = j['a']['#text']
            if j['@class'] == 'col_note':
                if 'a' in j.keys():
                    refseq_link = 'https://www.imgt.org' + j['a']['@href']
                    refseq_name = j['a']['#text']
                    lst.append(refseq_link)
                    lst.append(refseq_name)
                    break
        checker.append(allele_name)
        if len(lst) > 2:
            lst = lst[:2]
        if len(lst) < 2:
            continue
        dct['allele_link'] = allele_link
        dct['allele_name'] = allele_name
        dct['refseq_link'] = lst[0]
        dct['refseq_name'] = lst[1]
        rs.append(dct)
    rs2 = []
    for i in range(len(rs)-1):
        if rs[i]['allele_name'] != rs[i+1]['allele_name']:
            rs2.append(rs[i])
            
    with open(f'{path}/RefSeq/LinkTable/JSON/{name}.json', 'w') as fp:
        json.dump(rs2, fp)
        
def ParserRunner(lst):
    for i in lst:
        print(i)
        TableParser(i['name'])
        
def TableViewer():
    path = os.getcwd()
    
    f = urllib.request.urlopen("https://www.imgt.org/IMGTrepertoire/LocusGenes/listIG_TR/TR/human/Hu_TRgroup.html")
    file = f.read().decode("utf-8")
    soup = BeautifulSoup(file, 'lxml')
    xmlDict = xmltodict.parse(soup.table.prettify())
    lst = []
    for i in xmlDict['table']['tr'][1:]:
        if type(i['th']['a']) == list:
            link = i['th']['a'][0]['@href']
            name = i['th']['a'][0]['#text']
        else:
            link = i['th']['a']['@href']
            name = i['th']['a']['#text']
        lst.append({
        'link': 'https://www.imgt.org'+link,
        'name': name,
        })

    with open(f'{path}/RefSeq/RefGroups.json', 'w') as fp:
        json.dump(lst, fp)
    return lst
        

In [148]:
ParserRunner(TableViewer())

{'link': 'https://www.imgt.org/IMGTrepertoire/index.php?section=LocusGenes&repertoire=genetable&species=human&group=TRAC', 'name': 'TRAC'}
{'link': 'https://www.imgt.org/IMGTrepertoire/index.php?section=LocusGenes&repertoire=genetable&species=human&group=TRAJ', 'name': 'TRAJ'}
{'link': 'https://www.imgt.org/IMGTrepertoire/index.php?section=LocusGenes&repertoire=genetable&species=human&group=TRAV', 'name': 'TRAV'}
{'link': 'https://www.imgt.org/IMGTrepertoire/index.php?section=LocusGenes&repertoire=genetable&species=human&group=TRBC', 'name': 'TRBC'}
{'link': 'https://www.imgt.org/IMGTrepertoire/index.php?section=LocusGenes&repertoire=genetable&species=human&group=TRBD', 'name': 'TRBD'}
{'link': 'https://www.imgt.org/IMGTrepertoire/index.php?section=LocusGenes&repertoire=genetable&species=human&group=TRBJ', 'name': 'TRBJ'}
{'link': 'https://www.imgt.org/IMGTrepertoire/index.php?section=LocusGenes&repertoire=genetable&species=human&group=TRBV', 'name': 'TRBV'}
{'link': 'https://www.imgt.

In [108]:
checker = ['']
rs = []
for i in xmlDict['table']['tr'][2:]:
    dct = {}
    for j in i['td']:
        if j['@class'] == 'allele_note':
            lst = []
            allele_link = 'https://www.imgt.org' + j['a']['@href'].replace(' ', '%20')
            allele_name = j['a']['#text']
            #print(allele_link, allele_name)
        if j['@class'] == 'col_note':
            if 'a' in j.keys():
                refseq_link = 'https://www.imgt.org' + j['a']['@href']
                refseq_name = j['a']['#text']
                lst.append(refseq_link)
                lst.append(refseq_name)
                break
    checker.append(allele_name)
    if len(lst) > 2:
        lst = lst[:2]
    #if allele_name == checker[len(checker)-1]:
    dct['allele_link'] = allele_link
    dct['allele_name'] = allele_name
    dct['refseq_link'] = lst[0]
    dct['refseq_name'] = lst[1]
    rs.append(dct)
    #for k in lst:
rs2 = []
for i in range(len(rs)-1):
    if rs[i]['allele_name'] != rs[i+1]['allele_name']:
        rs2.append(rs[i])


In [100]:
len(rs)

203

In [31]:
lst

[13,
 13,
 13,
 13,
 13,
 2,
 3,
 13,
 13,
 13,
 13,
 13,
 2,
 3,
 13,
 13,
 2,
 3,
 2,
 2,
 3,
 3,
 13,
 13,
 2,
 13,
 13,
 13,
 13,
 13,
 13,
 3,
 13,
 2,
 13,
 13,
 3,
 4,
 13,
 13,
 13,
 13,
 4,
 13,
 13,
 13,
 4,
 13,
 3,
 4,
 13,
 13,
 13,
 2,
 3,
 13,
 2,
 2,
 3,
 2,
 2,
 3,
 3,
 3,
 2,
 13,
 2,
 3,
 13,
 2,
 13,
 13,
 3,
 4,
 3,
 4,
 13,
 13,
 3,
 13,
 13,
 13,
 13,
 3,
 4,
 13,
 3,
 4,
 13,
 2,
 3,
 13,
 2,
 3,
 13,
 2,
 3,
 2,
 3,
 13,
 3,
 13,
 13,
 13,
 3,
 2,
 3,
 3,
 13,
 13,
 3,
 13,
 13,
 13,
 3,
 3,
 3,
 3,
 13,
 13,
 3,
 3,
 13,
 13,
 3,
 3,
 3,
 13,
 13,
 3,
 3,
 2,
 13,
 13,
 2,
 3,
 13,
 13,
 13,
 3,
 3,
 3,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 2,
 13,
 13,
 13,
 13,
 13,
 2,
 3,
 2,
 3,
 3,
 13,
 13,
 13,
 13,
 2,
 3,
 13,
 13,
 13,
 13,
 3,
 2,
 3,
 2,
 3,
 13,
 13,
 13,
 13,
 2,
 3,
 13,
 2,
 3,
 3,
 13,
 3,
 2,
 3,
 3,
 13,
 3,
 3,
 2,
 3,
 13,
 13,
 3,
 2,
 3,
 13,
 2,
 3,
 2,
 3,
 3,
 13,
 13,
 3,
 2,
 3,
 13,
 13,
 2,
 13,
 